In [45]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [85]:
start_date = '2022-09-02'
end_date = '2022-09-11'

In [86]:
code = f"""
with customers as (
select 
    run_date,
    link_lifetime_stage as ls,
    case when (link_recency_segment  not in ('RECENT','STATIONARY') or link_lifetime_stage in ('HANDHOLDING','UNKNOWN'))  then 'Non_Allocator' 
    else 'Allocator' end as ls_stage,
    customer_id
from
    datasets.iallocator_customer_segments
where 
    run_date>=date_format(cast('{start_date}' as date)-interval '1' day,'%Y-%m-%d') 
    and run_date<=date_format(cast('{end_date}' as date),'%Y-%m-%d')
    and taxi_lifetime_last_ride_city='Kolkata'
)

select
    case 
    when day_of_week(date_parse(run_date,'%Y-%m-%d'))= 1 then 'Monday'
    when day_of_week(date_parse(run_date,'%Y-%m-%d'))= 2 then 'Tuesday'
    when day_of_week(date_parse(run_date,'%Y-%m-%d'))= 3 then 'Wednesday'
    when day_of_week(date_parse(run_date,'%Y-%m-%d'))= 4 then 'Thursday'
    when day_of_week(date_parse(run_date,'%Y-%m-%d'))= 5 then 'Friday'
    when day_of_week(date_parse(run_date,'%Y-%m-%d'))= 6 then 'Saturday'
    else 'Sunday' end as day
    ,run_date,
    count(distinct case when status_flag = 'Allocator_Upgrade' then customer_id end) as allocator_upgrade_users,
    count(distinct case when status_flag = 'Allocator_Downgrade' then customer_id end) as allocator_downgrade_users,
    count(distinct case when status_flag = 'Same' then customer_id end) as allocator_same_users

from
(
select
    *,
    case 
    when ls_stage='Allocator' and prev_day_ls='Non_Allocator' then 'Allocator_Upgrade'
    when ls_stage='Non_Allocator' and prev_day_ls='Allocator' then 'Allocator_Downgrade' 
    else 'Same' 
    end as status_flag
from
    (
    select 
        run_date,
        customer_id,
        ls_stage,
        lag(ls_stage) over(partition by customer_id order by run_date ASC) as prev_day_ls
    from
        customers
    )
)
--where status_flag !='Same'
group by 1,2
order by 2

"""

In [87]:
df_code = pd.read_sql(code, conn)
df_code

,day,run_date,allocator_upgrade_users,allocator_downgrade_users,allocator_same_users
0,Thursday,2022-09-01,0,0,1555152
1,Friday,2022-09-02,2858,3885,1549825
2,Saturday,2022-09-03,3167,3318,1551563
3,Sunday,2022-09-04,2651,1515,1555390
4,Monday,2022-09-05,3074,2021,1556021
5,Tuesday,2022-09-06,2754,3063,1556640
6,Wednesday,2022-09-07,2936,2993,1557000
7,Thursday,2022-09-08,2775,2901,1559408
8,Friday,2022-09-09,2800,1592,1561878
9,Saturday,2022-09-10,2801,2622,1562270


In [92]:
df_code['allocator_upgrade_users'].sum()

27728

In [93]:
df1 = df_code[df_code['run_date'] > '2022-09-04']
df1

,day,run_date,allocator_upgrade_users,allocator_downgrade_users,allocator_same_users
4,Monday,2022-09-05,3074,2021,1556021
5,Tuesday,2022-09-06,2754,3063,1556640
6,Wednesday,2022-09-07,2936,2993,1557000
7,Thursday,2022-09-08,2775,2901,1559408
8,Friday,2022-09-09,2800,1592,1561878
9,Saturday,2022-09-10,2801,2622,1562270
10,Sunday,2022-09-11,1912,2693,1563622


In [94]:
df1['allocator_upgrade_users'].sum()

19052

In [90]:
df_code['allocator_downgrade_users'].sum()

26603

In [91]:
df_code['allocator_same_users'].sum()

17128769